In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances
import random

In [2]:
articles = pd.read_csv('cleanDataArticles', delimiter=',', header=None)
tables = pd.read_csv('cleanDataTables', delimiter=',', header=None)

In [3]:
formattedTables = tables.iloc[:,:].values
formattedArticles = articles.iloc[:,:].values

In [4]:
corpusTables = []

for tables in formattedTables:
    
    corpusTables.append(str(tables[4]))

In [13]:
corpusArticlesID = []
corpusArticlesText = []

for i in range(10):
    
    randomArticle = random.choice(formattedArticles)
    
    corpusArticlesID.append(randomArticle[0])
    corpusArticlesText.append(str(randomArticle[1]))

In [31]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer='word')
tablesByCharacteristics = vectorizer.fit_transform(corpusTables)

In [32]:
articlesByCharacteristics = vectorizer.transform(corpusArticlesText)

In [8]:
def getTotalTablesByArticle(idArticle):

    countTopTables = 0

    for tables in formattedTables:
    
        if tables[0] == int(idArticle):
        
             countTopTables = countTopTables + 1
    
    #print(str(idArticle) + '-' + str(countTopTables))
    
    return countTopTables

In [9]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
    
        #index = distanceVector.index(topkDistance)
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [10]:
def getAccuracy(idRankedTables, idQueryGoal):

    totalRanquedTables = len(idRankedTables)

    match = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            match = match + 1

    accuracy = match / totalRanquedTables
    
    return accuracy

In [33]:
accuracy = []

totalSelectedArticles = int(articlesByCharacteristics.shape[0])

for i in range(totalSelectedArticles):
    
    distanceVector = pairwise_distances(articlesByCharacteristics[i], tablesByCharacteristics, metric='cosine')
    
    idQueryGoal = int(corpusArticlesID[i])
    
    countTopTables = getTotalTablesByArticle(idQueryGoal)
    
    topK = nsmallest(countTopTables, distanceVector[0])
    
    idRankedTables = getIdRankedTalbes(topK,distanceVector[0])
    
    #print(idRankedTables)
    
    accuracy.append(getAccuracy(idRankedTables,idQueryGoal))

In [50]:
accuracy

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0]

In [35]:
corpusArticlesID

[331039,
 707815,
 9962207,
 33957668,
 7973225,
 1108415,
 1749093,
 7408658,
 246185,
 23624157]